In [1]:
import sys
sys.path.append("../code/")
from common import APPLIANCES_ORDER




In [2]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
tensor = np.load('../1H-input.npy')

In [4]:
def create_subset_dataset(tensor):
    t_subset = tensor[:, :, 180:194, :]
    all_indices = np.array(list(range(320)))
    for i in range(1, 7):
        valid_homes = pd.DataFrame(t_subset[:, i, :].reshape(320, 14*24)).dropna().index
        all_indices = np.intersect1d(all_indices, valid_homes)
    t_subset = t_subset[all_indices, :, :, :].reshape(52, 7, 14*24)
    
    # Create artificial aggregate
    t_subset[:, 0, :] = 0.0
    for i in range(1, 7):
        t_subset[:, 0, :] = t_subset[:, 0, :] + t_subset[:, i, :]
    # t_subset is of shape (#home, appliance, days*hours)
    return t_subset, all_indices

In [5]:
t_all, valid_homes = create_subset_dataset(tensor)
t_all.shape

(52, 7, 336)

In [6]:
t_all.shape

(52, 7, 336)

In [7]:
def my_objective(y_pred, y_true):
    with tf.name_scope(None):
        return tf.losses.absolute_difference(y_pred, y_true)

In [8]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
import keras
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
n_movies = 3
n_users=3
n_latent_factors=2

Using TensorFlow backend.
/Users/nipun/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
APPLIANCES_ORDER

['aggregate', 'hvac', 'fridge', 'mw', 'dw', 'wm', 'oven']

In [51]:
train_agg = (t_all[:30, 0, :].reshape(30*14, 24))/t_all[:30, 0, :].max()
train_appliance = {}
test_appliance = {}
for appliance_num, appliance in enumerate(APPLIANCES_ORDER[1:]):
    train_appliance[appliance] = (t_all[:30, appliance_num+1, :].reshape(30*14, 24))/t_all[:30, 0, :].max()
    test_appliance[appliance] = t_all[30:, appliance_num+1, :].reshape(22*14, 24)






test_hvac = t_all[30:, 1, :].reshape(22*14, 24)
test_fridge = t_all[30:, 2, :].reshape(22*14, 24)

test_mw = t_all[30:, 3, :].reshape(22*14, 24)



test_agg = (t_all[30:, 0, :].reshape(22*14, 24))/t_all[:30, 0, :].max()

In [52]:
(model.predict(test_agg))

array([[ 0.07049746,  0.06813432,  0.07180402, ...,  0.06689776,
         0.05803183,  0.00872594],
       [ 0.06524456,  0.01594201,  0.00788268, ...,  0.10759436,
         0.03686026,  0.06834097],
       [ 0.01433881,  0.01724856,  0.07083076, ...,  0.10798766,
         0.13157904,  0.07804683],
       ..., 
       [ 0.17790517,  0.17655863,  0.09754188, ...,  0.16173653,
         0.18087827,  0.4173319 ],
       [ 0.25839847,  0.27371386,  0.22064829, ...,  0.17044914,
         0.4498792 ,  0.24323307],
       [ 0.22868428,  0.21941172,  0.16621615, ...,  0.18822099,
         0.4606916 ,  0.29861513]], dtype=float32)

In [59]:
pred_appliance = {}
num_iterations_dictionary = {'hvac':240,'fridge':500,'mw':250,'dw':250,'oven':250, 'wm':300}
for appliance in APPLIANCES_ORDER[1:2]:
    print(appliance)
    print("*"*20)
    np.random.seed(0)
    from keras.layers.merge import Subtract, Minimum
    from keras import regularizers
    agg_input = keras.layers.Input(shape=[24],name='Aggregate')
    appliance_dense_1 = keras.layers.Dense(units=40,name='Appliance-layer-1',activation='relu')(agg_input)
    dropout = keras.layers.Dropout(rate=0.1,name='Droput-Appliance')(appliance_dense_1)




    out = keras.layers.Dense(units=24,name='Appliance-output',activation='relu')(dropout)
    out = Minimum(name='Clip-to-agg')([out, agg_input])


    model = keras.Model(agg_input, out)
    model.compile('adam','mean_absolute_error')
    model.fit(train_agg, train_appliance[appliance], epochs=num_iterations_dictionary[appliance], validation_split=0.1)
    pred_appliance[appliance] = model.predict(test_agg)*t_all[:30, 0, :].max()



hvac
********************
Train on 378 samples, validate on 42 samples
Epoch 1/240
378/378 [==============================] - 1s 1ms/step - loss: 0.1333 - val_loss: 0.1261
Epoch 2/240
378/378 [==============================] - 0s 50us/step - loss: 0.1037 - val_loss: 0.0943
Epoch 3/240
378/378 [==============================] - 0s 55us/step - loss: 0.0790 - val_loss: 0.0713
Epoch 4/240
378/378 [==============================] - 0s 51us/step - loss: 0.0617 - val_loss: 0.0565
Epoch 5/240
378/378 [==============================] - 0s 50us/step - loss: 0.0540 - val_loss: 0.0494
Epoch 6/240
378/378 [==============================] - 0s 55us/step - loss: 0.0509 - val_loss: 0.0479
Epoch 7/240
378/378 [==============================] - 0s 56us/step - loss: 0.0479 - val_loss: 0.0452
Epoch 8/240
378/378 [==============================] - 0s 51us/step - loss: 0.0412 - val_loss: 0.0370
Epoch 9/240
378/378 [==============================] - 0s 53us/step - loss: 0.0349 - val_loss: 0.0309
Epoch 10/240

378/378 [==============================] - 0s 53us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 80/240
378/378 [==============================] - 0s 48us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 81/240
378/378 [==============================] - 0s 51us/step - loss: 0.0255 - val_loss: 0.0279
Epoch 82/240
378/378 [==============================] - 0s 47us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 83/240
378/378 [==============================] - 0s 52us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 84/240
378/378 [==============================] - 0s 46us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 85/240
378/378 [==============================] - 0s 49us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 86/240
378/378 [==============================] - 0s 48us/step - loss: 0.0256 - val_loss: 0.0278
Epoch 87/240
378/378 [==============================] - 0s 47us/step - loss: 0.0255 - val_loss: 0.0278
Epoch 88/240
378/378 [==============================] - 0s 55us/step - loss: 0.0255 - 

378/378 [==============================] - 0s 53us/step - loss: 0.0253 - val_loss: 0.0280
Epoch 158/240
378/378 [==============================] - 0s 52us/step - loss: 0.0252 - val_loss: 0.0280
Epoch 159/240
378/378 [==============================] - 0s 54us/step - loss: 0.0254 - val_loss: 0.0280
Epoch 160/240
378/378 [==============================] - 0s 46us/step - loss: 0.0253 - val_loss: 0.0280
Epoch 161/240
378/378 [==============================] - 0s 48us/step - loss: 0.0253 - val_loss: 0.0280
Epoch 162/240
378/378 [==============================] - 0s 47us/step - loss: 0.0253 - val_loss: 0.0280
Epoch 163/240
378/378 [==============================] - 0s 49us/step - loss: 0.0253 - val_loss: 0.0279
Epoch 164/240
378/378 [==============================] - 0s 56us/step - loss: 0.0253 - val_loss: 0.0279
Epoch 165/240
378/378 [==============================] - 0s 48us/step - loss: 0.0253 - val_loss: 0.0280
Epoch 166/240
378/378 [==============================] - 0s 49us/step - loss: 

Epoch 235/240
378/378 [==============================] - 0s 56us/step - loss: 0.0249 - val_loss: 0.0277
Epoch 236/240
378/378 [==============================] - 0s 56us/step - loss: 0.0250 - val_loss: 0.0277
Epoch 237/240
378/378 [==============================] - 0s 57us/step - loss: 0.0249 - val_loss: 0.0278
Epoch 238/240
378/378 [==============================] - 0s 49us/step - loss: 0.0249 - val_loss: 0.0279
Epoch 239/240
378/378 [==============================] - 0s 55us/step - loss: 0.0250 - val_loss: 0.0278
Epoch 240/240
378/378 [==============================] - 0s 57us/step - loss: 0.0249 - val_loss: 0.0280


In [32]:
with open('per-appliance.pdf','wb') as f:
    f.write(model_to_dot(model,  show_shapes=True, show_layer_names=True, rankdir='HB').create(prog='dot', format='pdf'))



In [60]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Aggregate (InputLayer)          (None, 24)           0                                            
__________________________________________________________________________________________________
Appliance-layer-1 (Dense)       (None, 40)           1000        Aggregate[0][0]                  
__________________________________________________________________________________________________
Droput-Appliance (Dropout)      (None, 40)           0           Appliance-layer-1[0][0]          
__________________________________________________________________________________________________
Appliance-output (Dense)        (None, 24)           984         Droput-Appliance[0][0]           
__________________________________________________________________________________________________
Clip-to-ag

In [61]:
from sklearn.metrics import mean_absolute_error
mae = {}
for appliance in APPLIANCES_ORDER[1:]:
    try:
        mae[appliance] = mean_absolute_error(test_appliance[appliance], pred_appliance[appliance])
    except Exception as e:
        print(e)
    
    

'fridge'
'mw'
'dw'
'wm'
'oven'


In [62]:
test_appliance['hvac']

array([[  284.06668091,   260.91665649,   280.2166748 , ...,
          243.7666626 ,   235.9833374 ,     0.        ],
       [  240.8666687 ,     0.        ,     0.        , ...,
          451.3999939 ,   115.3666687 ,   279.68331909],
       [    8.21666622,     0.        ,   270.54998779, ...,
          321.93331909,   523.45001221,   307.18331909],
       ..., 
       [  824.68334961,   819.2166748 ,   300.8500061 , ...,
          690.88336182,   783.04998779,  2006.68334961],
       [ 1167.61669922,  1257.66662598,   925.9666748 , ...,
          736.93334961,  2178.05004883,  1140.5       ],
       [  968.13336182,   959.18334961,   721.79998779, ...,
          785.43334961,  2122.94995117,  1417.83337402]])

In [63]:
pred_appliance['hvac']

array([[  352.5166626 ,   340.69998169,   359.04998779, ...,
          334.5166626 ,   290.18331909,    43.63333511],
       [  326.25      ,    79.71666718,    39.41666412, ...,
          538.01660156,   184.31665039,   341.7333374 ],
       [   71.69999695,    86.25      ,   354.18331909, ...,
          539.98327637,   657.95001221,   390.2666626 ],
       ..., 
       [  889.59997559,   882.86669922,   487.75      , ...,
          808.75      ,   904.4666748 ,  2086.83325195],
       [ 1292.09997559,  1368.68322754,  1103.33337402, ...,
          852.31671143,  2249.58325195,  1216.26660156],
       [ 1143.51660156,  1097.15002441,   831.14996338, ...,
          941.18334961,  2303.64990234,  1493.20007324]], dtype=float32)

In [64]:
pd.Series(mae)

hvac    138.096256
dtype: float64

In [65]:
pd.Series(mae)

hvac    138.096256
dtype: float64

In [66]:
pred_hvac.shape

NameError: name 'pred_hvac' is not defined

In [67]:
pred_hvac = model.predict(test_agg)


from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(pred_hvac, test_fridge))
print(mean_absolute_error(pred_hvac, test_agg))




88.7577573748
0.00213249505944


In [282]:
(pd.DataFrame(pred_hvac) - pd.DataFrame(test_agg)).max()

0     1.907349e-06
1    -3.877686e+00
2    -9.150000e+00
3     0.000000e+00
4     0.000000e+00
5    -9.633333e+00
6     0.000000e+00
7     2.861023e-06
8    -8.683333e+00
9     9.536743e-07
10    0.000000e+00
11    9.536743e-07
12   -9.583333e+00
13   -9.516666e+00
14   -4.711666e+01
15    3.099442e-06
16   -4.685000e+01
17    9.536743e-07
18   -7.310000e+01
19   -7.350000e+01
20   -4.180000e+01
21    0.000000e+00
22   -9.616667e+00
23   -9.533334e+00
dtype: float64

In [1]:
pd.Series(test_mw[1, :]).plot(label='GT')
#pd.Series(test_agg[1, :]).plot(label='GT')


pd.Series(model.predict(test_agg[1:2])[0, :24]).plot(label='Pred')




plt.legend()

NameError: name 'pd' is not defined